# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [216]:
# Your solution goes here
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [250]:
def imdb_moviesearch(title_type,start_date,end_date,user_rating):
    imdb_movies = {}
    item = 0
    url = f"https://www.imdb.com/search/title/?title_type={title_type}&release_date={start_date}-01-01,{end_date}-12-31&user_rating={user_rating}&lang=en"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept_Language" : "en-US,en;q=0.9"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content)
    movies = soup.find_all("div", attrs = {"class":"ipc-metadata-list-summary-item__c"})
    for movie in movies:
        try:
            moviepos = movie.find_all("h3", attrs = {"class":"ipc-title__text"})[0].get_text().split(" ")[0]
        except:
            moviepos = "N/A"
        try:
            movietitle = movie.find_all("h3", attrs = {"class":"ipc-title__text"})[0].get_text().split(" ")[1:]
        except:
            movietitle = "N/A"
        try:
            movieyear = movie.find_all("span", attrs = {"class":"sc-b189961a-8 kLaxqf dli-title-metadata-item"})[0].get_text()
        except:
            movieyear = "N/A"
        try:
            movierate = movie.find_all("span", attrs = {"class":"ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating"})[0].get_text().split()[0]
        except:
            movierate= "N/A"
        try:
            movieruntime = movie.find_all("span", attrs = {"class":"sc-b189961a-8 kLaxqf dli-title-metadata-item"})[1].get_text()
        except:
            movieruntime = "N/A"
        try:
            moviedesc = movie.find_all("div", attrs = {"class":"ipc-html-content-inner-div"})[0].get_text()
        except:
            moviedesc = "N/A"
        try:
            movievotes = movie.find_all("span", attrs = {"class":"ipc-rating-star--voteCount"})[0].get_text().split()[0].replace("(","").replace(")","")
        except:
            movievotes = "N/A"
        try:
            moviecert = movie.find_all("span", attrs = {"class":"sc-b189961a-8 kLaxqf dli-title-metadata-item"})[2].get_text()
        except:
            moviecert = "N/A"
        try:
            moviemeta = movie.find_all("span", attrs = {"class":"sc-b0901df4-0 bcQdDJ metacritic-score-box"})[0].get_text()
        except:
            moviemeta = "N/A"
        imdb_movies[item] = {"position" : moviepos,
                        "title" : " ".join(movietitle),
                        "year" : movieyear,
                        "runtime" : movieruntime,
                        "description" : moviedesc,
                        "rating" : movierate,
                        "votes" : movievotes,
                        "meta" : moviemeta,
                        "certificate" : moviecert
                        }
        item+= 1
        df = pd.DataFrame(imdb_movies).T
    return df

In [251]:
imdb_moviesearch("feature",1995,1996,8)

,position,title,year,runtime,description,rating,votes,meta,certificate
0,1.,Seven - 7 Pecados Mortais,1995,2h 7m,"Two detectives, a rookie and a veteran, hunt a...",8.6,1.8M,65,M/16
1,2.,Os Suspeitos do Costume,1995,1h 46m,The sole survivor of a pier shoot-out tells th...,8.5,1.1M,76,M/16
2,3.,Heat - Cidade Sob Pressão,1995,2h 50m,A group of high-end professional thieves start...,8.3,719K,76,M/16
3,4.,Trainspotting,1996,1h 33m,"Renton, deeply immersed in the Edinburgh drug ...",8.1,725K,83,M/18
4,5.,Braveheart: O Desafio do Guerreiro,1995,2h 58m,Scottish warrior William Wallace leads his cou...,8.3,1.1M,68,M/12
5,6.,Fargo,1996,1h 38m,Minnesota car salesman Jerry Lundegaard's inep...,8.1,726K,88,M/16
6,7.,Casino,1995,2h 58m,"In Las Vegas, two best friends - a casino exec...",8.2,564K,73,M/16
7,8.,Toy Story: Os Rivais,1995,1h 21m,A cowboy doll is profoundly threatened and jea...,8.3,1.1M,96,M/6
8,9.,Antes do Amanhecer,1995,1h 41m,A young man and woman meet on a train in Europ...,8.1,339K,78,M/12
9,10.,12 Macacos,1995,2h 9m,"In a future world devastated by disease, a con...",8.0,646K,75,M/12


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [159]:
# Your solution goes here